## Word2Vec & Node2Vec

In this notebook, I will provide some theoretical and technical overview of the word2vec and node2vec embedding methods. 

### 1. Word2Vec - Skip-gram and Negative Sampling

#### <b>Goal</b>: Given a text corpus, maximize the probability, $p(c|w)$, for this corpus, where $c$ is the context given a word, $w$. 

* More formally: $$ \arg \max_{\theta} \prod_{w \in \text{Text}} [\ \prod_{c \in C(w)} p(c | w; \theta)]\ $$ where $C(w)$ is the set of context for word $w$. 
    * Alternative formal definition: $$ \arg\max_{\theta} \prod_{(w, c) \in D} p(c|w; \theta) $$ where $D$ is the set of all word and context from the text. 

#### Parameterization of skip-gram model in Neural Network

* Define the softmax function for $p(c|w; \theta)$: $$ p(c|w; \theta) = \frac{e^{v_c \cdot v_w}}{\sum_{c' \in C} e^{v_c' \cdot v_w} } $$ where $v_c \text{ and } v_w$ are vector representations of $c \text{ and } w$ respectively, and $C$ is the set of all available contexts.

* The parameter we are trying to learn here are $v_c$ and $v_w$. For $w \in V, c \in C, i \in {1, ..., d}$, the total number of parameters is $|C| \times |V| \times d$. 

* To achieve the goal of maximizing the probability function, $\arg\max_{\theta} \prod_{(w, c) \in D} p(c|w; \theta)$, we take the log of the product and change the softmax representation of the probability into a summation of dot products between $v_c, v_w$: 
    $$ \arg\max_\theta \sum_{(w,c) \in \theta} \log p(c|w) = \sum_{(w,c) \in D} (\log e^{v_c \cdot v_w} - \log \sum_{c' \in C} e^{v_{c'} \cdot v_w}) $$

* <b>Assumption</b>: maximizing this objective function will result in good embeddings $v_w \forall w \in V$, i.e. similar words will have similar vectors. 

* Usually, to lower the computational cost of the above objective, we use hierarchical softmax. 

#### Negative Sampling 

* Negative sampling is based on the skip-gram model, but it uses a different objective function. 

* The idea behind this starts with considering the probability distribution of whether the pair $(w, c)$ is from the training dataset $D$; we note this probability as $p(D = 1 | w,c)$. Conversely, we have  $p(D = 0 | w,c) = 1 -  p(D = 1 | w,c)$ for the probability that the pair $(w,c)$ isn't from the training dataset. 

* Now, the parameters in $\theta$ controls the distribution $p(D = 1| w,c; D)$. This changes our goal to maximize the probability that all of the observations are from the training dataset: 
    $$ \arg\max_{\theta} \prod_{(w,c) \in D} p(D = 1 | w,c; \theta) \\ = \arg\max_\theta \sum_{(w,c) \in D} \log p(D = 1 | w,c; \theta)$$
    - We do the log trick again to change the product into a summation

* Again, using the softmax function, we define the probability of the pair of data coming from the training set as: $$ p(D = 1| w,c; \theta) = \frac{1}{1 + e^{-v_c \cdot v_w}}$$. Now we can rewrite the objective function as: $$ \arg\max_{\theta} \sum_{(w,c) \in D} \log \frac{1}{1 + e^{-v_c \cdot v_w}} $$. 

* <b>Issue</b>: We have a trivial solution. If we set the embedding parameter in $\theta$, $v_c$ and $v_w$ the same, i.e. $v_c = v_w$, and $v_c \cdot v_w = K \forall v_c, v_w$, and $K$ is a "large enough number" (in practice, "large enough" means $K \approx 40$). The objective function need some mechanism preventing it from arriving at the trivial solution, which is all embedding vectors being the same, by disallowing some $(w,c)$ combinations; this is the main mechanism of negative sampling. 

* One way is to present the model with $(w,c)$ pairs that are not in the data. We can do this by generating the set $D'$ with random $(w,c)$ pairs, and assuming they are all 'not from the training dataset'. Now the objective function have an extra term for the negative cases: 
    $$
    \arg\max_\theta \prod_{(w,c)\in D} p(D = 1| w,c; \theta) \prod_{(w,c) \in D'} p(D = 0| w,c; \theta) \\
    = \arg\max_\theta \prod_{(w,c)\in D} p(D = 1| w,c; \theta) \prod_{(w,c)\in D'} 1-p(D = 1| w,c; \theta) \\
    = \arg\max_\theta \sum_{(w,c)\in D} \log p(D = 1| w,c; \theta) \sum_{(w,c)\in D'} \log(1-p(D = 1| w,c; \theta)) \\
    = \arg\max_\theta \sum_{(w,c)\in D} \log(\frac{1}{1+e^{-v_c \cdot v_w}}) \sum_{(w,c)\in D'} \log (1-\frac{1}{1+e^{-v_c \cdot v_w}}) \\
    = \arg\max_\theta \sum_{(w,c)\in D} \log(\frac{1}{1+e^{-v_c \cdot v_w}}) \sum_{(w,c)\in D'} \log(\frac{1}{1+e^{v_c \cdot v_w}})  \\
    $$

* To simplify notation, let's write $\sigma(x) = \frac{1}{1+e^{-x}}$: 
    $$
    \arg\max_\theta \sum_{(w,c)\in D} \log(\frac{1}{1+e^{-v_c \cdot v_w}}) \sum_{(w,c)\in D'} \log(\frac{1}{1+e^{v_c \cdot v_w}})  \\
    = \arg\max_\theta \sum_{(w,c)\in D} \log(\sigma(v_c \cdot v_w)) \sum_{(w,c)\in D'} \log(\sigma(-v_c \cdot v_w))
    $$


#### Defining Contexts

* In general, the context of a word, $w_i$, is defined by the sentence it is in, $L: {w_1, w_2, ..., w_n}$ with $n$ words, and the parameter $k$. $k$ is the size of the window around the word $w_i$. So context for word $w$ is defined as:  
$$
    C(w) = {w_{i-k}, w_{i-(k-1)}, ..., w_{i-1}, w_{i+1}, ..., w_{i+k}}
$$

* <b>Dynamic Window size</b>: as the name suggests the window size is dynamic, and the parameter $k$ is the maximal windows size. For each word in the corpus, a window size of $k'$ is sampled uniformly from $1, ..., k$. 

* <b>Effect of subsampling and rare-word pruning</b>: In the word2vec implementation, words appearing less than ```min-count``` times are not considered as either words or contexts. In addition, frequent words, threshold by the ```sample``` parameter, are down-sampled; these words are removed from the text before generating contexts. This has the effect of increasing the effective window size for certain words. There are two explanations: 
    1. Frequent words are less informative. Sub-sampling the word improves the quality of the resulting embedding by removing noise.
    2. The effective window size for frequent words grow, including context-words which are both content-full and far away from the focus word, thus making the similarities more topical.  


### 2. Node2Vec